**3.2 - Apply text preprocessing techniques and TF-IDF for feature extraction of the first 200 comments**

In [1]:
!pip install underthesea

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import string

pd.set_option('display.max_colwidth',120) 
df = pd.read_csv("./data/data.csv",encoding='utf-8')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,comment
0,Đến cuối cùng thì chiếc CUP lớn nhất và quan trọng nhất đối với MU chính là chiếc CUP trong lòng NHM: đầu mùa nhuộm ...
1,Không phải fan ATM nhưng thấy thắng MU là mình rất thích 🤣🤣🤣🤣
2,Ở Simone luôn có nhiệt huyết và độ quái từ khi còn là cầu thủ. Có thể thấy Simone khi là HLV thì luôn làm cho các cầ...
3,"Đúng là câu nói hằng năm với MU “đầu mùa ăn 6 , giữa mùa ăn 3 , cuối mùa còn cái nịt”. Và thế là cuối cùng với MU “C..."
4,"Chúc mừng Aletico Marid , họ là những ng xứng đáng , MU chơi v là hay rồi nhưng chỉ kém may mắn trong những cơ hội ..."


In [3]:
len(df)

200

**Tiền xử lý dữ liệu**

In [4]:
# Chuyển từ in hoa thành in thường
df['lowercase'] = df['comment'].str.lower()
df['lowercase'].head()

0    đến cuối cùng thì chiếc cup lớn nhất và quan trọng nhất đối với mu chính là chiếc cup trong lòng nhm: đầu mùa nhuộm ...
1                                                              không phải fan atm nhưng thấy thắng mu là mình rất thích 🤣🤣🤣🤣
2    ở simone luôn có nhiệt huyết và độ quái từ khi còn là cầu thủ. có thể thấy simone khi là hlv thì luôn làm cho các cầ...
3    đúng là câu nói hằng năm với mu “đầu mùa ăn 6 , giữa mùa ăn 3 , cuối mùa còn cái nịt”. và thế là cuối cùng với mu “c...
4    chúc mừng aletico marid , họ là những  ng xứng đáng , mu chơi v là hay rồi nhưng chỉ kém may mắn trong những cơ hội ...
Name: lowercase, dtype: object

In [5]:
# Xóa bỏ kí tự đặc biệt và số
df['re_char'] = df['lowercase'].str.replace('[^\w\s]','')
df['re_num'] = df['re_char'].str.replace('\d','')
df['re_num'].head()

<ipython-input-5-c22f18f3d58d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['re_char'] = df['lowercase'].str.replace('[^\w\s]','')
<ipython-input-5-c22f18f3d58d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['re_num'] = df['re_char'].str.replace('\d','')


0    đến cuối cùng thì chiếc cup lớn nhất và quan trọng nhất đối với mu chính là chiếc cup trong lòng nhm đầu mùa nhuộm đ...
1                                                                  không phải fan atm nhưng thấy thắng mu là mình rất thích 
2    ở simone luôn có nhiệt huyết và độ quái từ khi còn là cầu thủ có thể thấy simone khi là hlv thì luôn làm cho các cầu...
3    đúng là câu nói hằng năm với mu đầu mùa ăn   giữa mùa ăn   cuối mùa còn cái nịt và thế là cuối cùng với mu chiếc cúp...
4    chúc mừng aletico marid  họ là những  ng xứng đáng  mu chơi v là hay rồi nhưng chỉ kém may mắn trong những cơ hội tấ...
Name: re_num, dtype: object

In [6]:
# Tách từ
from underthesea import word_tokenize
df['word_token'] = df['re_num'].apply(lambda x: word_tokenize(x))
df['word_token'].head(10)

0    [đến, cuối cùng, thì, chiếc, cup, lớn, nhất, và, quan trọng, nhất, đối với, mu, chính, là, chiếc, cup, trong, lòng, ...
1                                                      [không, phải, fan, atm, nhưng, thấy, thắng, mu, là, mình, rất, thích]
2    [ở, simone, luôn, có, nhiệt huyết, và, độ, quái, từ, khi, còn, là, cầu thủ, có thể, thấy, simone, khi, là, hlv, thì,...
3    [đúng, là, câu, nói, hằng, năm, với, mu, đầu, mùa, ăn, giữa, mùa, ăn, cuối, mùa, còn, cái, nịt, và, thế là, cuối cùn...
4    [chúc mừng, aletico, marid, họ, là, những, ng, xứng đáng, mu, chơi, v, là, hay, rồi, nhưng, chỉ, kém, may mắn, trong...
5                  [chúc mừng, atletico, chúc mừng, các, bạn, giành, vé, đi, tiếp, các, bạn, xứng đáng, hơn, mu, rất, nhiều]
6                                         [hàng, thủ, rất, ổn định, trận, nào, cũng, thủng lưới, đều đều, phong độ, quá, ổn]
7    [elanga, sút, cận, thành, trúng, đầu, oblak, quá, đáng, tiếc, felix, ghi bàn, nhưng, trước, đó, marcos, llorente, đã...


In [7]:
#stop word
with open('./data/stopwords.txt', 'r', encoding='UTF-8') as f:
    stop_word = [s.strip() for s in f.readlines()]
for i in range(10):
    print(stop_word[i])

a lô
a ha
ai
ai ai
ai nấy
ai đó
alô
amen
anh
anh ấy


In [8]:
import copy
for i in range(len(df['word_token'])):
    word = copy.deepcopy(df['word_token'][i])
    for j in range(len(word)):
        if df['word_token'][i][j] in stop_word:
            word.remove(df['word_token'][i][j])      
    df['word_token'][i] = word
df['word_token'].head(10)

0    [cup, mu, cup, nhm, đầu, mùa, nhuộm, đỏ, trời, âu, mùa, nhuộm, đỏ, hang, sâu, ngõ, xúc động, cđv, mu, tri ân, hlv, d...
1                                                                                                      [fan, atm, thắng, mu]
2          [simone, nhiệt huyết, độ, quái, cầu thủ, simone, hlv, cầu thủ, nể, sợ, giúp, cầu thủ, thi đấu, đông, đòi hỏi, đi]
3                                                    [câu, hằng, mu, đầu, mùa, mùa, mùa, nịt, mu, cúp, hâm mộ, cúp, quý giá]
4    [chúc mừng, aletico, marid, ng, xứng đáng, mu, v, kém, may mắn, tấn công, đi, tiếp, ta, trở lại, thời, hoàng kim, mã...
5                                                       [chúc mừng, atletico, chúc mừng, giành, vé, đi, tiếp, xứng đáng, mu]
6                                                                       [hàng, thủ, ổn định, trận, thủng lưới, phong độ, ổn]
7    [elanga, sút, cận, thành, trúng, đầu, oblak, tiếc, felix, ghi bàn, marcos, llorente, việt vị, griezmann, tạt, bóng, ...


**TF-IDF**

In [9]:
data_tf = ' '.join([str(i) for i in df['word_token']])
data_tf = [data_tf]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(data_tf)
tf_idf.toarray()

array([[0.01097048, 0.00548524, 0.00548524, 0.00548524, 0.00548524,
        0.00548524, 0.0548524 , 0.03291144, 0.00548524, 0.01097048,
        0.00548524, 0.01097048, 0.00548524, 0.00548524, 0.00548524,
        0.00548524, 0.00548524, 0.00548524, 0.00548524, 0.06033764,
        0.08776384, 0.00548524, 0.01097048, 0.01097048, 0.00548524,
        0.00548524, 0.01645572, 0.00548524, 0.01097048, 0.0274262 ,
        0.00548524, 0.03291144, 0.00548524, 0.00548524, 0.06582288,
        0.00548524, 0.06582288, 0.00548524, 0.00548524, 0.00548524,
        0.00548524, 0.00548524, 0.02194096, 0.19746864, 0.00548524,
        0.00548524, 0.00548524, 0.00548524, 0.00548524, 0.01097048,
        0.00548524, 0.01097048, 0.01097048, 0.00548524, 0.01097048,
        0.01097048, 0.01097048, 0.01097048, 0.00548524, 0.00548524,
        0.01097048, 0.00548524, 0.00548524, 0.00548524, 0.00548524,
        0.00548524, 0.00548524, 0.00548524, 0.00548524, 0.00548524,
        0.00548524, 0.01097048, 0.01097048, 0.00

In [11]:
name_tf = pd.DataFrame(tf_idf[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
result = name_tf.sort_values('TF-IDF', ascending=False)
print(result.head(25))

           TF-IDF
mu       0.564980
đội      0.285232
trận     0.202954
thủ      0.197469
bóng     0.197469
đá       0.191983
mùa      0.159072
mừng     0.153587
ko       0.126161
chúc     0.126161
thắng    0.120675
thua     0.115190
đấu      0.115190
fan      0.098734
đầu      0.093249
de       0.093249
atm      0.087764
thành    0.082279
công     0.082279
ronaldo  0.076793
gea      0.076793
giải     0.076793
đc       0.076793
hàng     0.076793
vệ       0.071308
